# Generación de tablas para Power BI (modelo global y diagnóstico)

Este notebook tiene como objetivo **materializar el output del modelo de devoluciones y del recomendador en tablas analíticas**, listas para ser consumidas directamente en Power BI.

No se entrena ningún modelo nuevo: aquí se **operacionaliza** lo ya construido, asegurando trazabilidad, coherencia y estabilidad entre datos, predicciones y métricas de negocio.

---

## 1. Predicciones globales a nivel item

A partir de los datos procesados del modelo (`X_train`, `X_test`) y sus índices asociados (`train_index`, `test_index`), se cargan:

- las features finales usadas por el modelo,
- el modelo XGBoost entrenado (`xgb_final.json`).

Se generan predicciones de probabilidad de devolución (`p_dev_global`) para **todo el universo histórico**, tanto train como test, y se construye una tabla única a nivel item:

- `item_id`
- `ticket_id`
- `customer_id`
- `fecha_compra`
- `devuelto_real`
- `p_dev_global`

Esta tabla se guarda como:

- `preds_global_item_level.csv`

y constituye la **base del diagnóstico global de riesgo**.

---

## 2. Diagnóstico económico global

Las predicciones se enriquecen con información real de los items (`items_devoluciones_ajustadas`), incorporando:

- precio neto,
- coste de devolución estimado,
- variables de producto, cliente y logística.

A partir de ello se calcula:

expected_cost_global = p_dev_global × coste_devolucion

Esto permite cuantificar, **antes de cualquier intervención**, el coste esperado de devoluciones por item.

El resultado se guarda como:

- `items_global_diagnostico.csv`

y sirve para:
- análisis exploratorio,
- mapas de riesgo,
- KPIs de coste esperado.

---

## 3. Agregaciones temporales para BI

Para facilitar el análisis temporal en Power BI, se generan tablas agregadas diarias:

### Por canal
- ventas
- número de tickets
- items
- clientes

→ `channel_daily.csv`

### Por categoría
- ventas
- tickets
- items
- clientes

→ `category_daily.csv`

### Por cliente
- presencia diaria del cliente

→ `customer_daily.csv`

Estas tablas permiten construir dashboards de evolución sin necesidad de cálculos complejos en BI.

---

## 4. Tabla item-level del modelo global (BI-friendly)

Se construye una tabla a nivel item que combina:

- identificadores (item, ticket, cliente),
- variable objetivo real,
- score del modelo (`p_dev_global`),
- proxy de coste (precio neto),
- coste esperado de devolución.

El resultado es:

- `items_model_global.parquet / csv`

Esta tabla está pensada como **fuente única de verdad** para análisis de riesgo y coste del modelo global en Power BI.

---

## 5. Tabla enriquecida final (modelo + negocio)

En un segundo paso, se genera una tabla **enriquecida**, uniendo:

- predicciones del modelo global,
- variables reales del item (producto, talla, cliente, logística),
- coste de devolución real,
- fechas de compra y devolución.

El merge se realiza de forma defensiva:
- validando claves,
- controlando duplicados,
- renombrando variables ambiguas (`devuelto_real`, `devuelto_items`).

El output final es:

- `items_model_global_enriched.parquet / csv`

Esta es la **tabla principal para Power BI**, a nivel item, desde la que se construyen:
- páginas de diagnóstico,
- análisis por categoría y producto,
- contexto económico del recomendador de tallas.

---

## 6. Relación con el recomendador de tallas

Este notebook es el **puente entre el modelo y el negocio**:

- el modelo global aporta el riesgo base (`p_dev_global`),
- el recomendador de tallas compara escenarios contra ese riesgo,
- Power BI consume ambos para medir impacto.

Gracias a esta estructura, es posible medir:

- ahorro por item,
- ahorro por intervención,
- share del coste total atacable por talla.

Sin este paso, el recomendador no sería auditable ni explicable desde negocio.

---

## 7. Conclusión

Este notebook no añade complejidad algorítmica, pero es **crítico para producción**. Garantiza que:

- las predicciones del modelo son reproducibles,
- los KPIs económicos son coherentes,
- el análisis en Power BI está alineado con la lógica del modelo.

Es el punto donde el proyecto deja de ser “modelado” y pasa a ser **sistema analítico usable**.



In [1]:
import pyarrow
print(pyarrow.__version__)

23.0.0


In [2]:
import xgboost as xgb
import pandas as pd
from pathlib import Path

BASE = Path("data/processed/devoluciones")

# cargar datasets
X_train = pd.read_parquet(BASE / "X_train.parquet")
X_test  = pd.read_parquet(BASE / "X_test.parquet")

train_idx = pd.read_parquet(BASE / "train_index.parquet")
test_idx  = pd.read_parquet(BASE / "test_index.parquet")

# cargar modelo
model = xgb.XGBClassifier()
model.load_model("modelos/devoluciones/xgb_final.json")

# predicciones
train_idx["p_dev_global"] = model.predict_proba(X_train)[:, 1]
test_idx["p_dev_global"]  = model.predict_proba(X_test)[:, 1]


In [3]:
preds_global = pd.concat(
    [train_idx, test_idx],
    axis=0,
    ignore_index=True
)

preds_global.head()


,fecha_compra,ticket_id,item_id,customer_id,devuelto,p_dev_global
0,2017-08-01,T000001,T000001-001,C000001,0,0.421737
1,2017-08-01,T000002,T000002-001,C000002,0,0.445007
2,2017-08-01,T000003,T000003-001,C000003,0,0.310177
3,2017-08-01,T000003,T000003-002,C000003,1,0.395241
4,2017-08-01,T000005,T000005-001,C000004,0,0.341808


In [4]:
preds_global.to_csv(
    "data/bi/preds_global_item_level.csv",
    index=False
)


In [5]:
print("Filas:", preds_global.shape[0])
print("Columnas:", preds_global.shape[1])
preds_global.head()


Filas: 905445
Columnas: 6


,fecha_compra,ticket_id,item_id,customer_id,devuelto,p_dev_global
0,2017-08-01,T000001,T000001-001,C000001,0,0.421737
1,2017-08-01,T000002,T000002-001,C000002,0,0.445007
2,2017-08-01,T000003,T000003-001,C000003,0,0.310177
3,2017-08-01,T000003,T000003-002,C000003,1,0.395241
4,2017-08-01,T000005,T000005-001,C000004,0,0.341808


In [6]:
preds_global.to_csv(
    "data/bi/preds_global_item_level.csv",
    index=False
)


# tabla maestra

In [7]:
import pandas as pd

items = pd.read_csv(
    "data/items_devoluciones_ajustadas.csv",
    encoding="utf-8-sig",
    parse_dates=["fecha_item", "fecha_devolucion"]
)

preds = pd.read_csv(
    "data/bi/preds_global_item_level.csv",
    encoding="utf-8-sig",
    parse_dates=["fecha_compra"]
)


C:\Users\PEDRO\AppData\Local\Temp\ipykernel_12044\1013978717.py:3: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  items = pd.read_csv(


In [8]:
df_global = items.merge(
    preds[["item_id", "p_dev_global"]],
    on="item_id",
    how="left"
)

In [9]:
df_global["expected_cost_global"] = (
    df_global["p_dev_global"] * df_global["coste_devolucion"]
)

df_global["venta_neta"] = df_global["precio_neto_unit"]


In [10]:
df_global.to_csv(
    "data/bi/items_global_diagnostico.csv",
    index=False,
    encoding="utf-8-sig"
)


# tablas bien

In [11]:
import pandas as pd
from pathlib import Path

df = pd.read_csv(
    "data/bi/items_global_diagnostico.csv",
    parse_dates=["fecha_item", "fecha_devolucion"],
    low_memory=False
)

# nos quedamos con la fecha día
df["dia"] = df["fecha_item"].dt.floor("D")

# normalizar canal/provincia por si acaso
df["canal_bi"] = (df["canal_norm"] if "canal_norm" in df.columns else df["canal"]).astype(str).str.lower().str.strip()
df["provincia_bi"] = (df["provincia_norm"] if "provincia_norm" in df.columns else df["provincia"]).astype(str).str.lower().str.strip()

print(df.shape)
df[["dia","ticket_id","item_id","customer_id","venta_neta","canal_bi","categoria"]].head()


(905445, 35)


,dia,ticket_id,item_id,customer_id,venta_neta,canal_bi,categoria
0,2017-08-01,T000001,T000001-001,C000001,90.25,online,Abrigo
1,2017-08-01,T000002,T000002-001,C000002,95.00,online,Abrigo
2,2017-08-01,T000003,T000003-001,C000003,26.00,online,Camiseta
3,2017-08-01,T000003,T000003-002,C000003,60.00,online,Sudadera
4,2021-02-14,T000004,T000004-001,C000003,51.00,online,Sudadera


In [12]:
channel_daily = (
    df.groupby(["dia","canal_bi"], dropna=False)
      .agg(
          ventas=("venta_neta","sum"),
          tickets=("ticket_id","nunique"),
          items=("item_id","count"),
          clientes=("customer_id","nunique"),
      )
      .reset_index()
      .rename(columns={"canal_bi":"canal"})
)

channel_daily.to_csv("data/bi/channel_daily.csv", index=False, encoding="utf-8-sig")
print("Guardado channel_daily:", channel_daily.shape)


Guardado channel_daily: (4352, 6)


In [13]:
category_daily = (
    df.groupby(["dia","categoria"], dropna=False)
      .agg(
          ventas=("venta_neta","sum"),
          tickets=("ticket_id","nunique"),
          items=("item_id","count"),
          clientes=("customer_id","nunique"),
      )
      .reset_index()
)

category_daily.to_csv("data/bi/category_daily.csv", index=False, encoding="utf-8-sig")
print("Guardado category_daily:", category_daily.shape)


Guardado category_daily: (21512, 6)


In [14]:
customer_daily = (
    df[["dia","customer_id"]]
      .dropna()
      .drop_duplicates()
      .sort_values(["dia","customer_id"])
)

customer_daily.to_csv("data/bi/customer_daily.csv", index=False, encoding="utf-8-sig")
print("Guardado customer_daily:", customer_daily.shape)


Guardado customer_daily: (440443, 2)


# tabla devoluciones

In [15]:
import os
import json
import pandas as pd
import xgboost as xgb

# Rutas (ajusta si tu proyecto tiene otras carpetas)
PATH_X_TRAIN = "data/processed/devoluciones/X_train.parquet"
PATH_X_TEST  = "data/processed/devoluciones/X_test.parquet"
PATH_I_TRAIN = "data/processed/devoluciones/train_index.parquet"
PATH_I_TEST  = "data/processed/devoluciones/test_index.parquet"

PATH_MODEL   = "modelos/devoluciones/xgb_final.json"

OUT_DIR      = "data/bi"
OUT_PARQUET  = os.path.join(OUT_DIR, "items_model_global.parquet")
OUT_CSV      = os.path.join(OUT_DIR, "items_model_global.csv")

FEATURES_JSON = "modelos/devoluciones/feature_columns.json"


def _ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def _load_and_check():
    X_train = pd.read_parquet(PATH_X_TRAIN)
    X_test  = pd.read_parquet(PATH_X_TEST)

    idx_train = pd.read_parquet(PATH_I_TRAIN)
    idx_test  = pd.read_parquet(PATH_I_TEST)

    # Check columnas iguales y en el mismo orden
    if list(X_train.columns) != list(X_test.columns):
        missing_in_test = set(X_train.columns) - set(X_test.columns)
        missing_in_train = set(X_test.columns) - set(X_train.columns)
        raise ValueError(
            "❌ X_train y X_test NO tienen exactamente las mismas columnas.\n"
            f"Missing in test: {missing_in_test}\n"
            f"Missing in train: {missing_in_train}\n"
            "Solución: alinear columnas antes de predecir."
        )

    # Check tamaños coinciden con index
    if len(X_train) != len(idx_train):
        raise ValueError(f"❌ X_train ({len(X_train)}) y train_index ({len(idx_train)}) no coinciden en filas.")
    if len(X_test) != len(idx_test):
        raise ValueError(f"❌ X_test ({len(X_test)}) y test_index ({len(idx_test)}) no coinciden en filas.")

    return X_train, X_test, idx_train, idx_test


def _predict_proba(model_path: str, X: pd.DataFrame) -> pd.Series:
    booster = xgb.Booster()
    booster.load_model(model_path)

    dmat = xgb.DMatrix(X, feature_names=list(X.columns))
    p = booster.predict(dmat)

    # Por si acaso, convertir a Series con mismo índice
    return pd.Series(p, index=X.index, name="p_dev_global")


def main():
    _ensure_dir(OUT_DIR)

    X_train, X_test, idx_train, idx_test = _load_and_check()

    # Guardar features para siempre (recomendado)
    _ensure_dir(os.path.dirname(FEATURES_JSON))
    with open(FEATURES_JSON, "w", encoding="utf-8") as f:
        json.dump(list(X_train.columns), f, ensure_ascii=False, indent=2)
    print(f"✅ Guardado {FEATURES_JSON} con {X_train.shape[1]} features")

    # Predicciones
    p_train = _predict_proba(PATH_MODEL, X_train)
    p_test  = _predict_proba(PATH_MODEL, X_test)

    # Construir tabla train y test con ids + predicciones
    train_out = idx_train.copy()
    train_out["p_dev_global"] = p_train.values

    test_out = idx_test.copy()
    test_out["p_dev_global"] = p_test.values

    # Concatenar (esto es tu tabla item-level diagnóstica)
    out = pd.concat([train_out, test_out], ignore_index=True)

    # Añadir coste base (para expected_cost)
    # Usamos precio_neto como proxy de "dinero en riesgo"
    # OJO: precio_neto está en X_train/X_test, no en index -> lo añadimos también
    # (para no perderlo, lo unimos en el mismo orden de filas)
    precio_neto_train = X_train["precio_neto"].reset_index(drop=True)
    precio_neto_test  = X_test["precio_neto"].reset_index(drop=True)
    out["precio_neto"] = pd.concat([precio_neto_train, precio_neto_test], ignore_index=True)

    # Coste base unitario (defensivo: no negativos)
    out["coste_base_unit"] = out["precio_neto"].clip(lower=0)

    # Coste esperado
    out["expected_cost_global"] = out["p_dev_global"] * out["coste_base_unit"]

    # Renombrar devuelto a devuelto_real (más claro en BI)
    if "devuelto" in out.columns:
        out = out.rename(columns={"devuelto": "devuelto_real"})

    # Reordenar columnas (BI-friendly)
    cols_first = [
        "fecha_compra",
        "ticket_id",
        "item_id",
        "customer_id",
        "devuelto_real",
        "p_dev_global",
        "precio_neto",
        "coste_base_unit",
        "expected_cost_global",
    ]
    cols_rest = [c for c in out.columns if c not in cols_first]
    out = out[cols_first + cols_rest]

    # Guardar
    out.to_parquet(OUT_PARQUET, index=False)
    out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

    print("✅ items_model_global creado")
    print("Parquet:", OUT_PARQUET)
    print("CSV:", OUT_CSV)
    print("Shape final:", out.shape)
    print(out.head(3))


if __name__ == "__main__":
    main()



✅ Guardado modelos/devoluciones/feature_columns.json con 83 features
✅ items_model_global creado
Parquet: data/bi\items_model_global.parquet
CSV: data/bi\items_model_global.csv
Shape final: (905445, 9)
  fecha_compra ticket_id      item_id customer_id  devuelto_real  \
0   2017-08-01   T000001  T000001-001     C000001              0   
1   2017-08-01   T000002  T000002-001     C000002              0   
2   2017-08-01   T000003  T000003-001     C000003              0   

   p_dev_global  precio_neto  coste_base_unit  expected_cost_global  
0      0.421614        90.25            90.25             38.050665  
1      0.442739        95.00            95.00             42.060214  
2      0.307088        26.00            26.00              7.984283  


In [16]:
import pandas as pd

base_path = "data/processed/devoluciones/"

files = [
    "X_train.parquet",
    "X_test.parquet",
    "y_train.parquet",
    "y_test.parquet",
    "train_index.parquet",
    "test_index.parquet"
]

for f in files:
    df = pd.read_parquet(base_path + f)
    print(f"\n📄 {f}")
    print("Shape:", df.shape)
    print("Columnas:")
    print(df.columns.tolist())



📄 X_train.parquet
Shape: (679083, 83)
Columnas:
['descuento', 'precio_neto', 'coste_bruto', 'margen', 'n_pedidos', 'n_items_comprados', 'altura_cm', 'peso_kg', 'anio_compra', 'mes_compra', 'edad_en_compra', 'antiguedad_cliente_dias', 'en_promocion', 'margen_relativo', 'desajuste_talla', 'desajuste_talla_abs', 'talla_extrema', 'bmi', 'compras_previas_cliente', 'devoluciones_previas_cliente', 'ratio_devoluciones_previas_cliente', 'ventas_previas_producto', 'devoluciones_previas_producto', 'ratio_devoluciones_previas_producto', 'precio_rel_cat', 'missing_provincia_cliente', 'missing_comunidad', 'missing_altura_cm', 'missing_peso_kg', 'missing_edad_en_compra', 'missing_antiguedad_cliente_dias', 'missing_desajuste_talla', 'missing_bmi', 'canal_online', 'comunidad_aragon', 'comunidad_asturias', 'comunidad_baleares', 'comunidad_canarias', 'comunidad_cantabria', 'comunidad_castilla y leon', 'comunidad_castilla-la mancha', 'comunidad_cataluna', 'comunidad_ceuta', 'comunidad_comunidad valencian

In [17]:
import os
import json
import pandas as pd
import xgboost as xgb

# =========================
# CONFIG
# =========================
PATH_X_TRAIN = "data/processed/devoluciones/X_train.parquet"
PATH_X_TEST  = "data/processed/devoluciones/X_test.parquet"
PATH_I_TRAIN = "data/processed/devoluciones/train_index.parquet"
PATH_I_TEST  = "data/processed/devoluciones/test_index.parquet"

PATH_MODEL   = "modelos/devoluciones/xgb_final.json"

# ✅ TU TABLA "ENRICHED" REAL (elige la que exista)
# Si está en /data directamente, prueba esto:
PATH_ITEMS_AJUSTADAS_PARQUET = "data/items_devoluciones_ajustadas.parquet"
PATH_ITEMS_AJUSTADAS_CSV     = "data/items_devoluciones_ajustadas.csv"

OUT_DIR = "data/bi"
OUT_PARQUET = os.path.join(OUT_DIR, "items_model_global_enriched.parquet")
OUT_CSV     = os.path.join(OUT_DIR, "items_model_global_enriched.csv")

FEATURES_JSON = "modelos/devoluciones/feature_columns.json"


# =========================
# HELPERS
# =========================
def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def load_items_ajustadas() -> pd.DataFrame:
    """Carga items_devoluciones_ajustadas desde parquet o csv."""
    if os.path.exists(PATH_ITEMS_AJUSTADAS_PARQUET):
        df = pd.read_parquet(PATH_ITEMS_AJUSTADAS_PARQUET)
        print(f"✅ Cargado {PATH_ITEMS_AJUSTADAS_PARQUET} | shape={df.shape}")
        return df

    if os.path.exists(PATH_ITEMS_AJUSTADAS_CSV):
        df = pd.read_csv(PATH_ITEMS_AJUSTADAS_CSV)
        print(f"✅ Cargado {PATH_ITEMS_AJUSTADAS_CSV} | shape={df.shape}")
        return df

    raise FileNotFoundError(
        "❌ No encuentro items_devoluciones_ajustadas.\n"
        f"Busqué:\n- {PATH_ITEMS_AJUSTADAS_PARQUET}\n- {PATH_ITEMS_AJUSTADAS_CSV}\n"
        "Solución: revisa el nombre exacto del archivo o su ruta."
    )


def load_and_check():
    X_train = pd.read_parquet(PATH_X_TRAIN)
    X_test  = pd.read_parquet(PATH_X_TEST)

    idx_train = pd.read_parquet(PATH_I_TRAIN)
    idx_test  = pd.read_parquet(PATH_I_TEST)

    # Columnas idénticas en train/test
    if list(X_train.columns) != list(X_test.columns):
        raise ValueError("❌ X_train y X_test no tienen las mismas columnas en el mismo orden.")

    # Tamaños coinciden con índices
    if len(X_train) != len(idx_train):
        raise ValueError(f"❌ X_train ({len(X_train)}) y train_index ({len(idx_train)}) no coinciden.")
    if len(X_test) != len(idx_test):
        raise ValueError(f"❌ X_test ({len(X_test)}) y test_index ({len(idx_test)}) no coinciden.")

    return X_train, X_test, idx_train, idx_test


def predict_proba(model_path: str, X: pd.DataFrame) -> pd.Series:
    booster = xgb.Booster()
    booster.load_model(model_path)

    dmat = xgb.DMatrix(X, feature_names=list(X.columns))
    p = booster.predict(dmat)
    return pd.Series(p, index=X.index, name="p_dev_global")


def keep_existing_cols(df: pd.DataFrame, cols: list[str]) -> list[str]:
    return [c for c in cols if c in df.columns]


# =========================
# MAIN
# =========================
def main():
    ensure_dir(OUT_DIR)
    ensure_dir(os.path.dirname(FEATURES_JSON))

    # 1) Cargar X + indices
    X_train, X_test, idx_train, idx_test = load_and_check()

    # 2) Guardar features (útil para siempre)
    with open(FEATURES_JSON, "w", encoding="utf-8") as f:
        json.dump(list(X_train.columns), f, ensure_ascii=False, indent=2)
    print(f"✅ Guardado {FEATURES_JSON} con {X_train.shape[1]} features")

    # 3) Predecir
    p_train = predict_proba(PATH_MODEL, X_train)
    p_test  = predict_proba(PATH_MODEL, X_test)

    # 4) Tabla diagnóstico básica (ids + prob)
    train_out = idx_train.copy()
    train_out["p_dev_global"] = p_train.values

    test_out = idx_test.copy()
    test_out["p_dev_global"] = p_test.values

    out = pd.concat([train_out, test_out], ignore_index=True)

    # Renombrar objetivo para BI
    if "devuelto" in out.columns:
        out = out.rename(columns={"devuelto": "devuelto_real"})

    # 5) Coste base y expected cost
    # Usamos el precio_neto que se usó en el modelo (feature)
    precio_neto_train = X_train["precio_neto"].reset_index(drop=True)
    precio_neto_test  = X_test["precio_neto"].reset_index(drop=True)
    out["precio_neto_model"] = pd.concat([precio_neto_train, precio_neto_test], ignore_index=True).clip(lower=0)

    out["expected_cost_global"] = out["p_dev_global"] * out["precio_neto_model"]

    # 6) Cargar tu tabla “enriched real” (items_devoluciones_ajustadas)
    items = load_items_ajustadas()

    if "item_id" not in items.columns:
        raise ValueError("❌ items_devoluciones_ajustadas no tiene 'item_id'.")

    # Evitar duplicados por item_id (merge seguro)
    dup = items["item_id"].duplicated().sum()
    if dup > 0:
        print(f"⚠️ Ojo: hay {dup} item_id duplicados en items_devoluciones_ajustadas. Me quedo con el primero.")
        items = items.drop_duplicates("item_id", keep="first")

    # 7) Seleccionar dimensiones útiles (las que existan)
    wanted_dims = [
        "item_id", "ticket_id", "customer_id",
        "canal", "provincia", "provincia_norm", "canal_norm", "zona_logistica",
        "fecha_item",
        "sku", "id_producto", "categoria", "color", "talla",
        "altura_cm", "peso_kg", "bmi",
        "pvp_unitario", "descuento_pct", "precio_neto_unit",
        "coste_bruto", "margen_unit",
        "coste_devolucion", "dias_hasta_devolucion", "fecha_devolucion",
        "devuelto"
    ]
    dim_cols = keep_existing_cols(items, wanted_dims)
    dims = items[dim_cols].copy()

    # Si en items también existe devuelto, lo renombramos para no confundir
    if "devuelto" in dims.columns:
        dims = dims.rename(columns={"devuelto": "devuelto_items"})

    # 8) Merge (out = many rows, dims = 1 por item_id)
    final = out.merge(dims, on="item_id", how="left", validate="many_to_one")

    # 9) Reordenar columnas (BI-friendly)
    first_cols = [
        "fecha_compra", "fecha_item",
        "ticket_id", "item_id", "customer_id",
        "sku", "id_producto", "categoria", "canal", "provincia",
        "color", "talla",
        "devuelto_real", "devuelto_items",
        "p_dev_global", "precio_neto_model", "precio_neto_unit",
        "expected_cost_global",
        "coste_devolucion"
    ]
    first_cols = [c for c in first_cols if c in final.columns]
    rest_cols = [c for c in final.columns if c not in first_cols]
    final = final[first_cols + rest_cols]

    # 10) Guardar
    final.to_parquet(OUT_PARQUET, index=False)
    final.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

    print("\n✅ items_model_global_enriched creado")
    print("Parquet:", OUT_PARQUET)
    print("CSV:", OUT_CSV)
    print("Shape final:", final.shape)

    # Checks rápidos
    if "categoria" in final.columns:
        pct_missing_cat = final["categoria"].isna().mean()
        print(f"ℹ️ % filas sin categoria tras merge: {pct_missing_cat:.2%}")

    print("\nPreview:")
    print(final.head(3))


if __name__ == "__main__":
    main()


✅ Guardado modelos/devoluciones/feature_columns.json con 83 features


C:\Users\PEDRO\AppData\Local\Temp\ipykernel_12044\2463886842.py:43: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH_ITEMS_AJUSTADAS_CSV)


✅ Cargado data/items_devoluciones_ajustadas.csv | shape=(905445, 29)

✅ items_model_global_enriched creado
Parquet: data/bi\items_model_global_enriched.parquet
CSV: data/bi\items_model_global_enriched.csv
Shape final: (905445, 33)
ℹ️ % filas sin categoria tras merge: 0.00%

Preview:
  fecha_compra  fecha_item      item_id          sku id_producto categoria  \
0   2017-08-01  2017-08-01  T000001-001   P005-NAV-L        P005    Abrigo   
1   2017-08-01  2017-08-01  T000002-001   P005-BEI-L        P005    Abrigo   
2   2017-08-01  2017-08-01  T000003-001  P002-BLU-XL        P002  Camiseta   

    canal provincia color talla  ...  zona_logistica  altura_cm  peso_kg  \
0  online   granada   LME     L  ...              Z2      184.2     71.2   
1  online  gipuzkoa   WHT     L  ...              Z1      182.3     75.4   
2  online    madrid   RED    XL  ...              Z1      189.6     94.5   

     bmi  pvp_unitario  descuento_pct  coste_bruto margen_unit  \
0  20.99          95.0          